### Create Table

In [3]:
# Define functions and load credentials
# import psycopg2
from config import database, user, password


# def execute_sql(query):
#     with psycopg2.connect(
#         host="localhost", database=database, user=user, password=password
#     ) as conn:

#         cur = conn.cursor()
#         cur.execute(query)
#         conn.commit()


def extract_enum_statement(s):
    if "enum" in s:
        ret = [i for i in s.split(" ") if "enum" in i][0]
        return ret
    else:
        return ""


def compose_enum_type(enum_statement, n):
    enteries = enum_statement.replace("enum", "")
    enum_type = f"CREATE TYPE enum_{n} as ENUM {enteries};\n"
    return enum_type


In [5]:
filename = "test.sql"

with open(filename, "r", encoding="utf-8") as f:

    sql_line = next(f, False).replace("`", '"')
    table_statement = ""
    enum_types_statements = ""
    num_enum_types = 0

    while sql_line:
        table_statement += sql_line

        if sql_line.startswith("CREATE TABLE"):
            print("Table found")

            while (";" not in sql_line) and sql_line:
                sql_line = next(f, False).replace("`", '"')

                if "enum" in sql_line:
                    enum_line = extract_enum_statement(sql_line)
                    enum_types_statements += compose_enum_type(
                        enum_line, num_enum_types
                    )
                    sql_line = sql_line.replace(enum_line, f"enum_{num_enum_types}")
                    num_enum_types += 1

                if "int" in sql_line:
                    int_subline = [i for i in sql_line.split(" ") if "int" in i][0]
                    sql_line = sql_line.replace(int_subline, "int")

                if "AUTO_INCREMENT," in sql_line:
                    sql_line = (
                        "  "
                        + [i for i in sql_line.split(" ") if len(i) > 0][0]
                        + " SERIAL,\n"
                    )

                if "COMMENT" in sql_line:
                    sql_line = sql_line.split(" COMMENT")[0] + ",\n"

                if "unsigned" in sql_line:
                    column_name = [i for i in sql_line.split(" ") if len(i) > 0][0]
                    sql_line = sql_line.replace(
                        "unsigned", f"check ({column_name} > 0)"
                    )

                if "COLLATE" in sql_line:
                    words = sql_line.split(" ")
                    for i, word in enumerate(words):
                        if word == "COLLATE":
                            del words[i + 1]
                            del words[i]
                            sql_line = " ".join(words)
                            break

                sql_line = sql_line.replace("datetime", "timestamp(0)")
                # constraints
                if "PRIMARY KEY" in sql_line:
                    column = sql_line.split("(")[1].split(")")[0]
                    for i in table_statement.split("\n"):
                        if column in i:
                            column_statement = i.replace(",", " PRIMARY KEY,")
                            table_statement = table_statement.replace(
                                i, column_statement
                            )
                            break
                    continue

                if "UNIQUE KEY" in sql_line:
                    column = sql_line.split("(")[1].split(")")[0]
                    for i in table_statement.split("\n"):
                        if column in i:
                            column_statement = i.replace(",", " UNIQUE,")
                            table_statement = table_statement.replace(
                                i, column_statement
                            )
                            break
                    continue

                if "FOREIGN KEY" in sql_line:
                    new_line = "  FOREIGN KEY" + sql_line.split("FOREIGN KEY")[1]
                    last_o_bracket = new_line.rfind("(")
                    last_c_bracket = new_line.rfind(")")
                    substring = new_line[last_o_bracket : last_c_bracket + 1]
                    new_line = new_line.replace(substring, "")
                    new_line = new_line.replace(' "', ' ("').replace('" ', '") ')
                    new_line = new_line.replace("REFERENCES", "REFERENCES parent_table")
                    sql_line = new_line
                elif "KEY" in sql_line:
                    continue

                if "KEY" in sql_line:
                    continue

                if ";" in sql_line:
                    table_statement = table_statement[:-2]
                    sql_line = ");"

                table_statement += sql_line

            table_statement = enum_types_statements + table_statement
            print("Table_statement done")
            break

        sql_line = next(f, False).replace("`", '"')

# execution is commented out because it is not needed for the demo. Print the statement to see the result
# execute_sql(table_statement)

print('\n',table_statement)


Table found
Table_statement done

 CREATE TYPE enum_0 as ENUM ('type_1','type_2');
CREATE TABLE "test_table" (
  "id" SERIAL PRIMARY KEY,
  "id_2" int NOT NULL UNIQUE,
  "sting_line" varchar(255) NOT NULL,
  "type" enum_0 NOT NULL);


### Insert data

In [6]:
# define function


def add_escape_char(s):
    words = s.split("'")
    for i, word in enumerate(words):
        if "\\" in word:
            if words[i - 1][-1] != "\\":
                words[i - 1] += "E"
    s = "'".join(words)
    return s


In [7]:
n = 0  # number of rows inserted

with open(filename, encoding="utf-8") as f:

    while True:
        sql_line = next(f, False)

        if not sql_line:
            break

        if sql_line.startswith("INSERT"):

            sql_line = sql_line.replace("`", "")
            if "\\" in sql_line:
                sql_line = add_escape_char(sql_line)

            try:
                # excutin is commented out because it is not needed for the demo. 
                # Print the statement to see the result
                
                # execute_sql(sql_line)
                print(sql_line)
                n += 1
            except Exception as e:  # If error occurs in inserting data write it to log file
                print(f"error at line {n}")
                print(e)
                with open("log_p2.txt", "a", encoding="utf-8") as log:
                    log.write(sql_line)

print(f"{n} lines loaded")

INSERT INTO orders VALUES (1,'2',E'McDonald\'s','type_1')

INSERT INTO orders VALUES (2,'1','KFC','type_2')
2 lines loaded
